In [98]:
%load_ext autoreload
%autoreload 2

# Shoestrings tutorial and cookbook

By [Allison Parrish](https://www.decontextualize.com/)

Shoestrings is a [Markov chain text generation](https://en.wikipedia.org/wiki/Markov_chain#Markov_text_generators) library for Python, with a particular focus on creative text generation. The library implements a simple ngram-based language model and uses the model to chain together predictions about what word will come next. Shoestrings is designed to be performant (using sparse matrices and numpy arrays internally) while remaining easy-to-use for beginners. Inspired by some common features of large language model generation libraries, Shoestrings includes implementations of some powerful features that are missing from other Markov chain text generation libraries out there, such as:

* [Beam search](https://en.wikipedia.org/wiki/Beam_search) and [softmax sampling](https://en.wikipedia.org/wiki/Softmax_function) with temperature
* Custom scoring criteria and stopping criteria for sequences (alongside the criterion of "most likely" according to the language model)
* Easy implementation of language- or application-specific tokenization
* Easy adaptation to tasks other than text generation

This notebook goes over some of the highlights of the library, starting with the `TextGenerator` class, which wraps some of the more advanced features of the library for beginners and other folks who just want to get some text generation going.

For a more general introduction to Markov chain text generation, see [this tutorial notebook I made for a different Markov chain generation library](https://github.com/aparrish/predictive-text-and-text-generation/blob/master/predictive-text-and-text-generation.ipynb). (Eventually I will incorporate much of this material into this notebook. All in due time, friends.)

## Warning: this is early-release software!

I'm putting this code on GitHub so I can use it in workshops, but it's not really ready for primetime yet. Please be aware that this is **barely alpha** software. Some stuff might be broken, and everything is subject to redesign and backwards incompatible changes at a moment's notice.

## Why Markov chain generation?

Let's say that the goal of making text with a computer is to create interesting new juxtapositions of bits of language that have never been produced before. Some goals alongside and complementary to this are to know where those juxtapositions came from, and to produce text that is meaningfully in dialogue with the sources and algorithms that brought that text into being. Markov chain text generation fits very well into an artistic process that has these goals—much better (in my opinion) than large language models. (You can [read more about my opinion here](https://posts.decontextualize.com/language-models-ransom-notes/).)

Additionally, Markov chain text generation works well on tiny corpora (as small as a single word) and can be built quickly on very modest hardware—quickly enough that the time it takes to "train" a Markov model from scratch on a reasonably-sized corpus (say, a novel-length text) is all but instantaneous. This variety of text generation fits better in a tight ideate-experiment-revise feedback loop that suits artistic creation.

## Working with a corpus

In the examples below, I'm going to use the text of Mary Shelley's *Frankenstein*. [The full text version that I'm using is available online.](https://github.com/aparrish/plaintext-example-files/blob/master/frankenstein.txt) You're welcome to use your own plain-text corpus instead!

## First steps

You'll need to install the Shoestrings library, which you can do with the following cell:

In [ ]:
import sys
!{sys.executable} -m pip install https://github.com/aparrish/shoestrings/archive/main.zip

Shoestrings has two major library dependencies: numpy and scipy. If you don't already have these installed, they should get installed as part of the installation process above.

For this notebook, I also suggest installing the [sentencex](https://pypi.org/project/sentencex/) library for simple and fast sentence tokenization:

In [ ]:
import sys
!{sys.executable} -m pip install sentencex

### Splitting the text into sentences

The following cell uses the `sentencex` library to open our source text and split it into a list of sentences. If you're using a different source file, put the file name in the call to `open()` below.

In [330]:
import sentencex
text = open("./frankenstein.txt").read()
sentences = sentencex.segment("en", text) # sentencex supports languages other than english! see the docs
# filter empty strings and fix mid-sentence line breaks
sentences = [item.replace("\n", " ") for item in sentences if len(item.strip()) > 0] 

I like to check the results of reading in a file by perusing the resulting data structure at random. Let's get a few random sentences from the source text:

In [331]:
import random
random.sample(sentences, 10)

['Margaret, if you had seen the man who thus capitulated for his safety, your surprise would have been boundless.',
 'In a few minutes after, I heard the creaking of my door, as if some one endeavoured to open it softly.',
 'Despair had indeed almost secured her prey, and I should soon have sunk beneath this misery.',
 'The astonishment which I had at first experienced on this discovery soon gave place to delight and rapture.',
 'He came.',
 'This, to my mother, was more than a duty; it was a necessity, a passion--remembering what she had suffered, and how she had been relieved--for her to act in her turn the guardian angel to the afflicted.',
 '"Then I fancy we have seen him, for the day before we picked you up we saw some dogs drawing a sledge, with a man in it, across the ice."',
 'She was thinner and had lost much of that heavenly vivacity that had before charmed me; but her gentleness and soft looks of compassion made her a more fit companion for one blasted and miserable as I was

### Generate some text

Now, we'll import the parts of the Shoestrings library we need to get started:

In [327]:
from shoestrings import TextGenerator
from shoestrings.tokenizers import SimpleWordTokenizer, SimpleCharacterTokenizer

The `TextGenerator` class is instantiated with three things: the length of the ngram that we want to use, a tokenizer, and a list of strings to use as the source text. We'll talk more about the specifics of tokenizers below; for now, I'm just going to pass an instance of the `SimpleWordTokenizer` class:

In [208]:
gen = TextGenerator(n=3, tokenizer=SimpleWordTokenizer(), source=sentences)

After creating the object, call its `.generate_one()` method to generate new stretches of text from the model:

In [210]:
gen.generate_one()

'Miserable himself that he, the snow that obstructed me and said some words in a country of eternal light?'

To generate a few lines, call this in a `for` loop and print:

In [211]:
for i in range(6):
    print(gen.generate_one())
    print()

Clerval spoke thus as we hurried through the cottage and prepared the food, and I was destined to endure their present hardships.

Strange and harrowing must be of use in undeceiving them.

TO Mrs. Saville, England

Could they turn from their sockets in attending him; he breathed with difficulty the words of Agatha were thrown, by insensible steps, not high, but she persisted, and do not feel for them on occasion to pass the night or hasten its conclusion by an eagerness which perpetually increased, I passed an hour in this peopled earth.

What freedom?

Beyond Cologne we descended to the existence of their miserable fare.



When generating text from the underlying language model, the `TextGenerator` object does its best to ensure that every text begins with an ngram that occurs at the beginning of a sentence somewhere in the corpus, and to ensure that every sentence ends at a sentence boundary (i.e., sentence-final punctuation like `.`, `!`, `?` and so forth.) 

For a character-based Markov chain rather than a word-based Markov chain, you can use `SimpleCharacterTokenizer` instead:

In [337]:
char_gen = TextGenerator(n=8, tokenizer=SimpleCharacterTokenizer(), source=sentences)

In [341]:
print(char_gen.generate_one())

All was called forth from the woods.


But we'll stick with the word-based generator for the rest of these examples.

### Temperature: the basics

There are a few parameters that you can play with right off the bat. I'll go into some detail explaining how these parameters work later, but for now you can experiment with them to see what differences they make to the output. The first parameter is `temperature`, which allows you to adjust the shape of the probabilities at each step of the prediction process. I'll explain this in more detail below, but the gist is that as the temperature gets lower (approaching zero), the model will tend to pick tokens with already high probability; as the temperature gets *higher*, the model distinguishes less and less between the tokens' probability in the underlying data. A few examples will demonstrate. First, here are a few low-temperature generations:

In [223]:
for i in range(6):
    print(gen.generate_one(temperature=0.05))
    print()

Poor Justine was the same time she gently deplored her own mind that the fiend should openly attack me. I am not so changeable as the sun had risen, he had been the favourite of her lover, instantly informed me of my father' s house.

Dear mountains! my own heart.

Yesterday the stranger.

Under the guidance of my own heart, I was unable to solve them.

As I spoke, rage sparkled in my own heart, but I was not, as I was unable to solve these questions, but I was unable to solve them.

Harmony was the same time, but I was unable to solve these questions, but I was unable to solve these questions, but I was not so selfish.



As you can see, at low temperatures (say, less than 0.5), the model tends to select common words and produce repetitive patterns, sometimes getting caught in "loops" for protracted periods. In some cases, it never predicts that the end of the sentence will arrive, and you see the model giving up and ceasing to adding tokens after the `max_tokens` limit (discussed below). At a high temperature, we find the opposite to be the case:

In [241]:
for i in range(4):
    print(gen.generate_one(temperature=5.0))
    print()

All, save me, until from the ocean he was, you will never consent."

Write, dearest Victor, when wearied by a merchantman now on its homeward voyage from Archangel; more fortunate than I became more attentive and interested; I began the creation; I will confide this tale and his friendship was of little happiness who did not participate in these words were legible in one occupation that I dated my creation.

Ay, stare if you obey me in my labour for the dreams of bliss that cannot be realized. What was I?

St. Bernard' s attention, and almost endless journey across the fields with a panegyric upon modern chemistry, the oak had disappeared from the grave- worms crawling in the impotence of despair. But this was to procure and promising me the list of several days.



Here the model seems to select more unusual turns of phrase and rarely gets caught in loops—instead, it tends to ramble on a bit. The default value for temperature is `1.0`, which causes the model to pick continuations at exactly the probability in which they occur in the source text. In practice, it's worthwhile to experiment with values around 1.0 (say, 0.35 up to 1.5, depending on your source text and ngram length) to find the sweet spot between repetition and unpredictability.

### Beam search: the basics

The second parameter worth talking about is `beam_width`, which is a parameter to the Shoestrings model's [beam search](https://en.wikipedia.org/wiki/Beam_search) algorithm. When the `beam_width` parameter is greater than 1, the `.generate_one()` function (and the other generation functions that we'll discuss below) internally keeps track of *several generated sequences at once*, instead of generating a single sequence. At each timestep, each of these sequences is evaluated for its combined probability, and the model keeps only the top `n` sequences (where `n` is equal to the `beam_width` parameter).

You can think of this a little bit like a "divide and conquer" strategy. If you lost your keys your keys and were trying to find them, one strategy would be to go to the room where you believe you are most likely to have left them, and then look in the piece of furniture in that room where you were most likely to have left them, and then open the drawer in the piece of furniture where you were most likely to have left them, etc. A potentially better strategy might be to ask a few friends over, and assign each friend to look in the first, second, third (etc) most likely rooms where your keys might be, and then they in turn would look in the first, second, third (etc) most likely pieces of furniture in each room, and so forth. That's sort of like how a beam search works: we're trying to find the most likely sequence of words in the apartment of the model's possibility space.

> *Pedants note*: the metaphor above describes a depth-first search strategy, while beam search is breadth-first. It's difficult to come up with real-life analogies for breadth-first search, sorry.

In essence, a beam search is a way of attempting to maximize the probability of *the entire sequence*, rather than simply maximizing the probability of each ngram within the sequence. The default value for `beam_width` is 1, meaning that no beam search is performed at all. In practice, you can sometimes get more "realistic" sounding results by increasing the `beam_width` parameter, at the cost of time and memory use. Let's give it a shot:

In [247]:
for i in range(4):
    print(gen.generate_one(beam_width=25))
    print()

Elizabeth saw even this last overwhelming event?

During this interval, one by one glimmering and seemingly ineffectual light. The modern masters promise very little and conversed in broken accents:" Unhappy man!

His countenance instantly assumed an aspect expressive of disgust and affright.

London was our darling and our pride!"



Hmm, well, okay, those don't really feel any different from normal generated sentences. Trust me, though, beam search is useful, and we'll see some better examples later on when we talk about custom scoring. I do find that beam search helps to "tame" some of the more outrageous word choices of high-temperature generation, however:

In [248]:
for i in range(4):
    print(gen.generate_one(beam_width=25, temperature=3.0))
    print()

Thus Elizabeth endeavoured to soothe me when others would have introduced some other topic than that which preceded it.

Life and death.

Vegetables and bread, cheese, milk, and a linen jacket being her only garb; her words pierce my heart.

Answer me, and taking his guitar, played some airs so entrancingly beautiful that they might not debar him from the shore.



If you call the `.generate()` method instead of `.generate_one()`, you'll get *all* of the sequences that resulted from the beam search, rather than just the most likely one. The results are sorted in order from most likely to least likely.

In [257]:
print(gen.generate(beam_width=8))

['Clouds hid the summits of its rage. Let your compassion be moved, and I ardently desired to divine.', 'Clouds hid the summits of its rage. Let your compassion be moved, and I ardently desired to divine.', 'Clouds hid the summits of its rage. Let your compassion be moved, and I ardently desired to divine.', 'Clouds hid the summits of its rage. Let your compassion be moved, and I ardently desired to divine.', 'Clouds hid the summits of its rage. Let your compassion be moved, and I ardently desired to divine.', 'Clouds hid the summits of its rage. Let your compassion be moved, and I ardently desired to divine.', 'Clouds hid the summits of its rage. Let your compassion be moved, and I ardently desired to plead, she replied that I was not splintered by the multitude of feelings which bore me onwards, like a mighty tide, overwhelmed every other circumstance of existence pass before me. I leave you, and I saw the mighty Alps, whose hands seem only made to a conclusion.', 'Clouds hid the sum

### Constraining the number of tokens

The `max_tokens` parameter tells the model to stop generating after the generated sequence has reached a particular number of tokens. This can be helpful when you want to ensure that you don't end up generating extremely long sequences, if the model happens to get caught in a loop. It can also be helpful for prototyping, if you don't want to spend a lot of time generating longer sequences just to check to see if an idea works.

In [254]:
for i in range(4):
    print(gen.generate_one(temperature=0.1, max_tokens=20))

Another storm enlightened Jura with faint flashes; and I was unable to solve them.
Heaven bless my beloved country; but I was not so wretched a condition.
Soft tears again bedewed my cheeks, and I was unable to solve them.
No; but I was unable to solve these questions, but I was not the master said,"


(Note that this does *not* guarantee that the sequences will end with appropriate punctuation or an end-of-sentence token.)

### Where generation begins

By default, the `TextGenerator` object's `.generate()` and `.generate_one()` methods start with a randomly chosen ngram that was found at the beginning of any sentence in the training data. The `start_string` parameter lets you pick a string to start with instead. This can be helpful when you want a bunch of different possible generations that begin with the same sequence, e.g.:

In [278]:
for i in range(4):
    print(gen.generate_one(start_string="She was"))
    print()

She was there to subdue me to speak, but it would be more than he pities me?

She was not then cold.

She was dressed in mourning, and now he instantly changed the subject of their victim and spared no pains or exertions on my misfortunes, I saw a wildness in my tastes for natural science.

She was senseless, and nature again assume the barren and bleak appearance it had been committed at the bottom of the kitchen stove.



It's important to note that the `start_string` argument *must occur somewhere in the source text*, and must be the same length in tokens as the model's ngram length minus one (e.g., for a word-tokenized model with ngram length 3, the `start_string` parameter must be two words). It would be fun if you could prompt a Markov chain text generator ChatGPT-style, but alas, it is not to be. If the tokens in the string do *not* occur in the source text, you'll get a key error:

In [279]:
print(gen.generate_one(start_string="Hey Frankie"))

KeyError: 'Hey'

### A simple way to modify scores

If you provide a `custom_scorer` parameter to the `.generate()` or `.generate_one()` function, the value of that parameter will be *invoked as a function* for each context and continuation under consideration during the generation process. The function call includes three parameters: the string of the continuation, the string of the entire generated sequence so far, and the probability already assigned to the token. If you're familar with this kind of thing, you can think of `custom_scorer` as a *callback* that allows you to modify the probability score of any continuation, using your own rule.

If you're not familiar with this kind of thing, then here's an example to make it a bit more concrete.

In [295]:
def show(token, sequence, score):
    print("sequence:", sequence)
    print("token:", token)
    print("score:", score)
    print()
    return score
gen.generate_one(custom_scorer=show, max_tokens=10)

sequence: Mine
token: has
score: 0.0

sequence: Mine has
token: been
score: 0.0

sequence: Mine has been
token: .
score: -2.5649493574615367

sequence: Mine has been
token: a
score: -2.5649493574615367

sequence: Mine has been
token: acted
score: -2.5649493574615367

sequence: Mine has been
token: consumed
score: -2.5649493574615367

sequence: Mine has been
token: discovered
score: -2.5649493574615367

sequence: Mine has been
token: done
score: -2.5649493574615367

sequence: Mine has been
token: dreadfully
score: -2.5649493574615367

sequence: Mine has been
token: hitherto
score: -2.5649493574615367

sequence: Mine has been
token: lying
score: -2.5649493574615367

sequence: Mine has been
token: my
score: -2.5649493574615367

sequence: Mine has been
token: passed
score: -2.5649493574615367

sequence: Mine has been
token: so
score: -2.5649493574615367

sequence: Mine has been
token: the
score: -2.5649493574615367

sequence: Mine has been lying
token: here
score: 0.0

sequence: Mine has b

'Mine has been lying here some days I have'

In this case, the `show()` function is being called every time the model attempts to evaluate the probability of a new token before adding it to the sequence. In my implementation of the `show()` function here, I'm printing out the parameters that are supplied to the function, so you can see how the process works. The `sequence` is the string of the sequence so far; the `token` is the string of a potential token to add next; the `score` is the probability score that the model has already assigned to that token.

#### Important math aside on log probabilities

The Shoestrings library uses *log probabilities* internally, as do many (if not most!) software libraries that deal with probabilities. The basic issue of working with probabilities is this. Let's say that we're calculating the probability of drawing the ace of hearts from a poker deck. The probability can be calculated as:

In [297]:
1 / 52

0.019230769230769232

... or about 2%. Now let's say that we're calculating the probability of drawing the ace of hearts from a (shuffled) poker deck twice in a row. That can be calculated as:

In [299]:
(1 / 52) * (1 / 52)

0.00036982248520710064

or about... what is that, three hundredths of a percent. If we go to *ten* times in a row, we have:

In [301]:
pow((1 / 52), 10) # one over fifty two to the tenth power

6.9177770887761845e-18

... which is scientific notation for about:

In [303]:
0.000000000000000006917777

6.917777e-18

... that is, `6.91777...` preceded by seventeen zeros. Not only are these numbers difficult to read, they're also difficult for computers to process—after a certain number of decimal places, it becomes difficult for computers to perform arithmetic accurately. Fortunately, we can convert very small numbers like these to a more convenient notation using the `log()` function:

In [307]:
import numpy as np
print(np.log(1 / 52))
print(np.log((1 / 52) * (1 / 52)))
print(np.log(pow((1 / 52), 10)))

-3.951243718581427
-7.902487437162854
-39.51243718581427


... and then retrieve the original number with the `exp()` function:

In [308]:
np.exp(-3.951243718581427) # or 1 / 52

0.019230769230769235

So one reason to use the *log* of the probability is so we have easier-to-read and easier-to-calculate numbers. The `log()` function also has another useful property, which is that $log(x \times y) = log(x) + log(y)$. This is useful because calculating probabilities of independent events—such as picking the same card from a shuffled poker deck twice, and picking the next token from a language model—consists of *multiplying together the probability of each event*, as we saw above with the example of drawing the ace of hearts twice in a row (i.e. $\frac{1}{52} \times \frac{1}{52}$). Because of the identity above, we can actually calculate this figure as an *addition* rather than *multiplication* by adding the results of the `log()` function together:

In [310]:
print((1/52)*(1/52)) # multiplying probabilities
print(np.exp(np.log(1/52) + np.log(1/52))) # adding log probabilities

0.00036982248520710064
0.0003698224852071008


(Yes, the results don't match after a bunch of digits, but that's because the computer uses slightly different algorithms for calculation in the first case versus the second case—the results of these operations are proven to be mathematically identical.)

Computers are *very* fast at adding numbers, and but not so fast at multiplying them, so using addition here can really speed up calculations (especially when you're calculating the probability of thousands or millions of events in a row, as is common in token prediction models).

**If none of that made any sense**, you're still okay. The short explanation is this:

* when a probability is represented as a log probability, a score of $0$ is equivalent to a probability of $1$
* as a probability gets smaller, the number of its log probability goes further negative (i.e., a log probability of $-10$ represents a probability lower than $-5$)
* if you absolutely need to know the more traditional probability of a log probability, you can convert it using `np.exp()`

#### Back to custom scorers

So back to what we were talking about: custom scoring. The probability given for each token is represented as a *log probability*. The text generation process will use whatever value our function returns as a *replacement* for the token's previously calculated log probability. That means that we can use this function to mess stuff up and boost particular tokens' probability (by adding to it) and/or diminishing the probability of others (by subtracting from it). Here's an example:

In [326]:
def my_boost(token, sequence, score):
    if token.startswith('a'): # you can put any expression that evaluates to True or False between `if` and `:`
        return 0
    else:
        return score
for i in range(4):
    print(gen.generate_one(custom_scorer=my_boost))
    print()

Chance-- or rather his sister the first time, and an awe which almost assured me that he and his dream was to all appearance dead.

St. Bernard' s account, but a type of me. No one can conceive a greater degree of certainty, I arrived at a convent at Leghorn; and another darkened and sometimes in the attempt until I felt as if all hell surrounded me told me that what I am a wretch."

Natural philosophy is the acquirement of knowledge along the precipitous sides of vast mountains of ice and should I dwell upon the beach.

Does it now only am I truly thank him.



What I've done here is tell the text generation function that any token that begins with `a` should have a log probability of zero (equal to a probability of $1$). Other tokens will have whatever probability had previously been assigned. You can see in the output that, indeed, there are an unnaturally large number of tokens that begin with `a` in the output!

You'll notice that not all of the sentences aggressively use `a`-initial words—this is simply happenstance based on luck of the draw. This is where beam search really shines! If we increase the width of the beam search, the model's generation function will search harder for sequences that match our criteria.

In [344]:
def my_boost(token, sequence, score):
    if token.startswith('a'):
        return 0
    else:
        return score
for i in range(4):
    print(gen.generate_one(custom_scorer=my_boost, beam_width=30))
    print()

Sweet and beloved as I am alone and miserable as I advanced perplexed me, and allured by the assurance of her acquittal."

Most of the affair.

All was silent and appears uneasy when anyone except myself enters his cabin.

Coleridge' s account, and arm themselves for my approach.



You can program any criteria you want in the `my_boost()` function. Here's another example, which tries to generate sequences with words no longer than five letters, by setting any word meeting those criteria to a probability of one, and reducing the log probability of others: (Upping the temperature a bit also helps in my experience.)

In [347]:
def my_boost(token, sequence, score):
    if len(token) <= 5:
        return 0
    else:
        return score - 5
for i in range(4):
    print(gen.generate_one(custom_scorer=my_boost, beam_width=30, temperature=1.5))
    print()

Uttering a few.

Think not, I found it was in the house which I, and alone.

Agatha, the sea.

We were, and did not for me. For a long pause of the deed.



Or how about generating sequences that don't include the word `the`:

In [350]:
def my_boost(token, sequence, score):
    if token == 'the':
        return score - 100
    else:
        return score
for i in range(4):
    print(gen.generate_one(custom_scorer=my_boost, beam_width=30, temperature=1.5))
    print()

Everything is related in them which would fully reward his toil and hazard.

Geneva, May 18th, 17--

Uttering a few lines in haste to say," Immediately upon your being taken ill, very dreadful."

Having paid his debts, therefore, to draw inexhaustible stores of affection and mutual misfortune.



The sky is the limit! Note that you're *not* guaranteed to always produce text that meets your criteria; in a Markov chain, it's possible that the *only* possible way to continue the chain is with (for example) the word `the`, and so the model has to select it, regardless of how low its probability score is. But with a large enough corpus, and for carefully selected tasks, it should work most of the time.

more stuff TK, but you can also look in the code documentation:

## Using the MarkovModel class directly

## Tokenizers

## Processors

## Stoppers